#Load Library 

In [1]:
## import all necessary packages
import json
import re
import pandas as pd

import nltk, string
from nltk.collocations import *

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from collections import defaultdict

import data_cleaning_hr as tc

import string
import happyfuntokenizing

#Load Data

In [74]:
with open('./data/tweets_training_clean.json','r') as f:
    tweet_df = DataFrame(json.load(f))

#Clean Data (Handle, URL, Emoticon Conversion)

In [3]:
tc.clean(tweet_df)
tweet_df.head()

293672 handles replaced
187643 urls replaced
:-*\(|\):  ->     😞 replaced   4600 times
</3        ->     💔 replaced      0 times
:-*\|      ->     😐 replaced     70 times
8\)        ->     😎 replaced     81 times
:-*>       ->     😆 replaced      0 times
:-*\)|\(:  ->     😃 replaced  12516 times
;-*\)      ->     😉 replaced   2688 times
:-*[PpbB]  ->     😛 replaced    962 times
=-*\)      ->     😀 replaced    174 times
:-*[/\\]   ->     😕 replaced   1571 times
:-*\*      ->     💋 replaced    238 times
:'\(       ->     😢 replaced    263 times
:-*D       ->     😀 replaced   1220 times
;-*[PpbB]  ->     😜 replaced    263 times
>:-*\(     ->     😠 replaced      0 times
:-*[Oo]    ->     😮 replaced    295 times
:o\)       ->     🐵 replaced      0 times
D:         ->     😧 replaced    160 times
<3         ->     ❤ replaced      0 times
ALL replaced 25101 times
834937 retweets processed


,id,lat,lng,text,timeStamp,user_id,retweet
0,383560784879833088,37.369106,-121.998494,"I'm at SEPTEMBER♥ (Sunnyvale, CA) url",Fri Sep 27 11:56:31 +0000 2013,47880944,NaN
1,385148713460453376,37.002203,-122.058052,oops. fell asleep in the library. class started at 2.,Tue Oct 01 21:06:23 +0000 2013,259008472,NaN
10,380062755639676928,37.438669,-121.879072,hdl you told me your birthday yourself 2chainz lol,Tue Sep 17 20:16:36 +0000 2013,630468267,NaN
100,384797380198928384,37.299621,-121.982024,She throw it back like Thursday,Mon Sep 30 21:50:19 +0000 2013,272108001,NaN
1000,383691693767143425,39.137889,-121.658646,Not ready for this exam,Fri Sep 27 20:36:43 +0000 2013,790458278,NaN


#Emoji Finder

In [4]:
try:
    # Wide UCS-4 build
    highpoints = re.compile(u'['
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\u2600-\u26FF\u2700-\u27BF]+', 
        re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    highpoints = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)

In [5]:
faces = re.compile(u'['
        u'\U0001F600-\U0001F64F]',
        re.UNICODE)
# Function that takes a list of text and return text that contains just faces
def just_face(text):
    return (faces.findall(text))
# Function that take a list of text and return text with just emojis
def just_emojis(text):
    return (highpoints.findall(text))
# Function that a list of text and return the number of emojis in the text.
def count_emojis(text):
    return len(highpoints.findall(text))

# Functions to check whether there's an emoji in the text, return 1 if true, 0 if false
def is_emoji(text):
    if highpoints.search(text):
        return 1
    else:
        return 0

    # Functions to check whether there's a face emoji in the text, return 1 if true, 0 if false
def is_face(text):
    if faces.search(text):
        return 1
    else:
        return 0

In [8]:
emoji_df["Emoji Count"] = emoji_df["text"].apply(count_emojis)
emoji_df["Text Length"] = emoji_df["text"].apply(lambda x: len(x))
emoji_df = emoji_df[['id', 'text', 'timeStamp', 'user_id', 'Emoji Count', 'Text Length']]
emoji_df.describe()

,id,user_id,Emoji Count,Text Length
count,2.082520e+05,2.082520e+05,208252.000000,208252.000000
mean,3.822541e+17,5.198651e+08,1.122659,53.161506
std,2.378889e+15,4.612050e+08,0.603226,33.193435
min,3.776523e+17,2.100000e+01,1.000000,1.000000
25%,3.801828e+17,1.499460e+08,1.000000,27.000000
50%,3.823091e+17,3.852509e+08,1.000000,46.000000
75%,3.843753e+17,7.546697e+08,1.000000,75.000000
max,3.863382e+17,1.934553e+09,70.000000,214.000000


# Exploring Emoji vs no Emoji (20% Text has Emojis, 15% has face)

###Functions to check whether it's emoji or face

In [73]:
tweet_df["is_emoji"] = tweet_df.text.apply(is_emoji)
tweet_df["is_face"] = tweet_df.text.apply(is_face)
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id,is_emoji,is_face
0,377652254096228352,37.446100,-121.883557,hdl hey checkout the website: url,Wed Sep 11 04:38:08 +0000 2013,224874450,0,0
1,377652255346159616,34.087406,-117.462604,hdl 😪,Wed Sep 11 04:38:08 +0000 2013,312179473,1,1
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,Wed Sep 11 04:38:10 +0000 2013,54351774,0,0
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,Wed Sep 11 04:38:11 +0000 2013,343219606,0,0
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by hdl @ St. John's Bar &amp; Grill — url,Wed Sep 11 04:38:12 +0000 2013,1569395935,0,0


In [25]:
tweet_df.describe()

,id,lat,lng,user_id,is_emoji,is_face
count,1.000000e+06,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000
mean,3.822416e+17,37.241980,-121.533430,4.961052e+09,0.208252,0.154146
std,2.370783e+15,1.333404,1.542418,4.503597e+12,0.406058,0.361089
min,3.776523e+17,12.983300,-170.296751,1.200000e+01,0.000000,0.000000
25%,3.801867e+17,37.320896,-122.350934,7.382110e+07,0.000000,0.000000
50%,3.822777e+17,37.656377,-122.043733,3.188787e+08,0.000000,0.000000
75%,3.843437e+17,37.790622,-121.785433,6.229686e+08,0.000000,0.000000
max,3.887526e+17,57.170120,77.583300,4.503597e+15,1.000000,1.000000


##Take a look at the imported tokenizer and test it

In [6]:
tok = happyfuntokenizing.TweetTokenizer(preserve_case=False)
samples = (
    u"RT @ #happyfuncoding: this is a typical Twitter tweet😖",
    u"😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth; other Web oddities can be an &aacute;cute <em class='grumpy'>pain</em> >:(",
    u"Yay my cat is cuddling🔫 with me tonight❤ +1 (800) 123-4567, (800) 123-4567, and 123-4567 are treated as words despite their whitespace."
    )

In [31]:
for s in samples:
        print("======================================================================")
        print(s)
        tokenized = tok.tokenize(s)
        print(list(tokenized))
        print("\n")

RT @ #happyfuncoding: this is a typical Twitter tweet😖
['rt', '@', '#happyfuncoding', ':', 'this', 'is', 'a', 'typical', 'twitter', 'tweet', '😖']


😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth; other Web oddities can be an &aacute;cute <em class='grumpy'>pain</em> >:(
['😂', '😂', '😂', 'rt', '@yours_truly3x', ':', 'bitch', 'brush', 'yoo', 'mouth', ';', 'other', 'web', 'oddities', 'can', 'be', 'an', 'ácute', "<em class='grumpy'>", 'pain', '</em>', '>:(']


Yay my cat is cuddling🔫 with me tonight❤ +1 (800) 123-4567, (800) 123-4567, and 123-4567 are treated as words despite their whitespace.
['yay', 'my', 'cat', 'is', 'cuddling', '🔫', 'with', 'me', 'tonight', '❤', '+1 (800) 123-4567', ',', '(800) 123-4567', ',', 'and', '123-4567', 'are', 'treated', 'as', 'words', 'despite', 'their', 'whitespace', '.']




#Add a new column that display just the emojis for the text

In [20]:
ex = 'hdl not funny! Sad 😞😭 bye money 💸. Now you have to drive me places Donna'
def textExtract(sent):
    return ''.join([word for word in sent if is_emoji(word) == 0])

def emojiExtract(sent):
    return [word for word in tok.tokenize(sent) if is_emoji(word) == 1]

textExtract(ex)

'hdl not funny! Sad  bye money . Now you have to drive me places Donna'

In [7]:
def emojiExtract(sent):
    return [word for word in tok.tokenize(sent) if is_emoji(word) == 1]

def textExtract(sent):
    return ''.join([word for word in sent if is_emoji(word) == 0])

def addTokenizedText(df):
    def textTokenized(sent):
        return [word for word in tok.tokenize(sent)]
    df['Tokenized'] = [textTokenized(word) for word in df.text]

def addEmojiCol(df):
    df['Emoji'] = [emojiExtract(word) for word in df.text]

def addText(df):
    df['only_Text'] = [textExtract(word) for word in df.text]

def addHashTag(df):
    df['only_HashTag'] = [re.findall(r"(#\w+)", word) for word in df.text]

In [71]:
def splitTextEmoji(df):
    def emojiExtract(sent):
        return [word for word in tok.tokenize(sent) if is_emoji(word) == 1]

    def textExtract(sent):
        return ''.join([word for word in sent if is_emoji(word) == 0])

    def addEmoji(df):
        df['only_Emoji'] = [emojiExtract(word) for word in df.text]

    def addText(df):
        df['only_Text'] = [textExtract(word) for word in df.text]
    
    addText(df)
    addEmoji(df)
    return

In [75]:
splitTextEmoji(tweet_df)

In [76]:
tweet_df.head()

,id,lat,lng,retweet,text,timeStamp,user_id,only_text,only_Emoji
0,383560784879833088,37.369106,-121.998494,None,"I'm at SEPTEMBER♥ (Sunnyvale, CA) url",Fri Sep 27 11:56:31 +0000 2013,47880944,"I'm at SEPTEMBER (Sunnyvale, CA) url",[♥]
1,385148713460453376,37.002203,-122.058052,None,oops. fell asleep in the library. class started at 2.,Tue Oct 01 21:06:23 +0000 2013,259008472,oops. fell asleep in the library. class started at 2.,[]
10,380511446926635008,37.376171,-122.030318,None,Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) url,Thu Sep 19 01:59:33 +0000 2013,777228427,Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) url,[]
100,379601897100812289,37.650087,-122.423051,None,I slept hella late anyways.,Mon Sep 16 13:45:19 +0000 2013,1104686922,I slept hella late anyways.,[]
1000,378303709446295552,37.776362,-122.281945,None,walking with my headphones on kinda high ~ ok ~ I'm feeling today 😎🙏,Thu Sep 12 23:46:47 +0000 2013,597925598,walking with my headphones on kinda high ~ ok ~ I'm feeling today,"[😎, 🙏]"


In [40]:
addEmojiCol(tweet_df)
addText(tweet_df)
addTokenizedText(tweet_df)
addHashTag(tweet_df)
addTokenizedText(tweet_df)
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id,retweet,Emoji,only_Text,Tokenized
0,377652254096228352,37.446100,-121.883557,hdl hey checkout the website: url,Wed Sep 11 04:38:08 +0000 2013,224874450,NaN,[],"[hdl, hey, checkout, the, website, :, url]","[hdl, hey, checkout, the, website, :, url]"
1,377652255346159616,34.087406,-117.462604,hdl 😪,Wed Sep 11 04:38:08 +0000 2013,312179473,NaN,[😪],[hdl],"[hdl, 😪]"
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,Wed Sep 11 04:38:10 +0000 2013,54351774,NaN,[],"[i, laugh, a, lot, with, that, line]","[i, laugh, a, lot, with, that, line]"
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,Wed Sep 11 04:38:11 +0000 2013,343219606,NaN,[],"[sons, of, anarchy, is, back, on, woop, woop]","[sons, of, anarchy, is, back, on, woop, woop]"
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by hdl @ St. John's Bar &amp; Grill — url,Wed Sep 11 04:38:12 +0000 2013,1569395935,NaN,[],"[drinking, a, fresh, squeezed, ipa, by, hdl, @, st, ., john's, bar, &, amp, ;, grill, —, url]","[drinking, a, fresh, squeezed, ipa, by, hdl, @, st, ., john's, bar, &, amp, ;, grill, —, url]"


# Word2Vec Trained Model with Twitter Corpus

In [34]:
import gensim
from gensim.models import Word2Vec

In [38]:
#emoji_model = gensim.models.Word2Vec(list(tweet_df.Tokenized))
# It might take some time to train the model. So, after it is trained, it can be saved as follows:
#emoji_model.save('emoji.embedding')
emoji_model = gensim.models.Word2Vec.load('emoji.embedding')

In [39]:
emoji_model.most_similar(positive = ['😂'], topn = 20)

[('ommmmmg', 0.7462140321731567),
 ('#dead', 0.6925972700119019),
 ('meagan', 0.6692696809768677),
 ('lmaooooooooooo', 0.6570378541946411),
 ('inseparable', 0.6502166986465454),
 ('#lrt', 0.6376644372940063),
 ('#playa', 0.6227041482925415),
 ('#dying', 0.5953817367553711),
 ('#sike', 0.5952829122543335),
 ('#coldworld', 0.5948764681816101),
 ('#blama', 0.5885862112045288),
 ('#icant', 0.5870974063873291),
 ('#workaholics', 0.5856915712356567),
 ('laffin', 0.5782320499420166),
 ('hahahahahahahahahahahahaha', 0.5772393345832825),
 ('. .. .', 0.5763115286827087),
 ('daaamn', 0.5759264826774597),
 ('omgosh', 0.5746556520462036),
 ('nikkah', 0.5687729716300964),
 ('gawwwd', 0.5650306344032288)]

In [233]:
len(tweet_df[tweet_df.text.str.contains(r'#bye')])

151

#How many Hashtags? 155979

In [454]:
tweet_df[tweet_df.only_HashTag.str.len() != 0].count(0)

id              155979
lat             155979
lng             155979
text            155979
timeStamp       155979
user_id         155979
is_emoji        155979
is_face         155979
Emoji           155979
only_Text       155979
only_HashTag    155979
Tokenized       155979
dtype: int64

#Language Cleaning

In [29]:
punctuation = string.punctuation
ex = ['“', '—', '’', ' ️', '️', '...', '”', '…', ' , , ,', '?', ' ', ' ⃣', '∞', '🆒']
for pun in [word for word in ex if word not in punctuation]:
    punctuation += pun
def isEnglish(list):
    try:
        [word.encode('ascii') for word in list if word not in punctuation]
    except Exception:
        return False
    else:
        return True

In [83]:
def cleanNonEnglish(df):
    temp = df.copy()
    text_list = df['only_text'].values
    english_Boolean = [isEnglish(sent) for sent in text_list]
    df = temp[english_Boolean]
    return temp[english_Boolean]

In [84]:
tweet_df_clean = cleanNonEnglish(tweet_df.iloc[0:10000])
len(tweet_df_clean)

9763

In [82]:
cleanNonEnglish(tweet_df.iloc[0:10000])
len(tweet_df)

800000

In [46]:
??pandas.DataFrame.drop

In [93]:
def apply_hashtag_split(_hashtag):
    # for each hashtag in list of hashtags, split on # and take second item
    item = [item.split('#')[1] for item in _hashtag if len(item) > 0]
    
    # regex pattern
    pattern = r'''([A-Z]{2,})      | (1) split on two caps or more, and only keep caps
                  ([A-Z]{1}[a-z]*)   (2) split on exactly one cap or more, and keep 
                                         trailing letters '''
    
    # loop through each item in list of hashtags
    final_hashtags = []
    for word in item:
        
        # if len of word is 0, then there is no hashtag
        if len(word) == 0:
            print("empty_hashtag")
            final_hashtags.append("empty_hashtag")
        
        # use (1)) regex: funciton lowercase, as "Treatlowercase" can be treated as lowercase
        elif word[0].isupper() and word[1:].islower():
            print("lower_forced: " + word + " : ", end="")
            print(infer_spaces(word.lower()))
            final_hashtags.append(infer_spaces(word.lower()))
        
        # use (1) regex: funciton lowercase
        elif word.islower():
            print("lower       :: " + word + " : ", end="")
            print(infer_spaces(word))
            final_hashtags.append(infer_spaces(word))
        
        # use (2) regex: customized uppercase
        else:
            print("upper       : " + word + " : ", end="")
            print(list(filter(None, re.split(pattern, word))))
            final_hashtags.append(list(filter(None, re.split(pattern, word))))
    return final_hashtags

In [87]:
addHashTag(tweet_df)
tweet_df

,id,lat,lng,retweet,text,timeStamp,user_id,only_text,only_Emoji,only_HashTag
0,383560784879833088,37.369106,-121.998494,None,"I'm at SEPTEMBER♥ (Sunnyvale, CA) url",Fri Sep 27 11:56:31 +0000 2013,47880944,"I'm at SEPTEMBER (Sunnyvale, CA) url",[♥],[]
1,385148713460453376,37.002203,-122.058052,None,oops. fell asleep in the library. class started at 2.,Tue Oct 01 21:06:23 +0000 2013,259008472,oops. fell asleep in the library. class started at 2.,[],[]
10,380511446926635008,37.376171,-122.030318,None,Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) url,Thu Sep 19 01:59:33 +0000 2013,777228427,Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) url,[],[]
100,379601897100812289,37.650087,-122.423051,None,I slept hella late anyways.,Mon Sep 16 13:45:19 +0000 2013,1104686922,I slept hella late anyways.,[],[]
1000,378303709446295552,37.776362,-122.281945,None,walking with my headphones on kinda high ~ ok ~ I'm feeling today 😎🙏,Thu Sep 12 23:46:47 +0000 2013,597925598,walking with my headphones on kinda high ~ ok ~ I'm feeling today,"[😎, 🙏]",[]
10000,379419855755046912,32.858170,-115.587649,None,#whitegirlprobs,Mon Sep 16 01:41:57 +0000 2013,94475205,#whitegirlprobs,[],[#whitegirlprobs]
100000,382371707580715008,38.391578,-121.998953,None,My teeth look hella fake in that picture👌,Tue Sep 24 05:11:33 +0000 2013,1654199208,My teeth look hella fake in that picture,[👌],[]
100001,382188203471409152,38.336622,-122.677975,None,i sometimes feel like college is like high school all over again 😑,Mon Sep 23 17:02:23 +0000 2013,162522832,i sometimes feel like college is like high school all over again,[😑],[]
100002,380086987174068224,37.704381,-121.004349,None,The bus is suppose to get here at 48 it's 53 bitch bitch bitch bitch bitch bitch bitch wtf,Tue Sep 17 21:52:54 +0000 2013,552455857,The bus is suppose to get here at 48 it's 53 bitch bitch bitch bitch bitch bitch bitch wtf,[],[]
100003,383024874991939584,37.244318,-121.954576,None,I'm gonna keep good grades this year url,Thu Sep 26 00:27:01 +0000 2013,827159958,I'm gonna keep good grades this year url,[],[]


In [90]:
tweet_df['only_HashTag'].apply(apply_hashtag_split())

TypeError: apply_hashtag_split() missing 1 required positional argument: '_hashtag'

In [95]:
def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""
    
    # Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return list(reversed(out))

In [96]:
apply_hashtag_split(['#whitegirlprobs'])

lower       :: whitegirlprobs : 

NameError: global name 'maxword' is not defined